In [1]:
import torchvision
from torch.utils.data import DataLoader

/home/sethupat/.conda/envs/ELIR-p39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import datasets
import transforms

def generate_dataloader(batch_size, csv, root):
    dataset = datasets.VideoDataset(csv,
                                    root,
                                    transform=torchvision.transforms.Compose([transforms.VideoFolderPathToTensor()]))

    return DataLoader(dataset,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=2)


def get_dataloader(batch_size, csv_train, root_train, csv_test, root_test):
    return {
        'train': generate_dataloader(batch_size, csv_train, root_train),
        'test': generate_dataloader(batch_size, csv_test, root_test)}

In [3]:
# os.chdir(os.path.join(os.getcwd(),'elearning-teaching-intervention-recommender'))

import torch
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Get the current device (GPU or CPU)
    current_device = torch.cuda.current_device()
    
    # Get the name of the current device
    device_name = torch.cuda.get_device_name(current_device)
    
    print(f"Using GPU: {device_name}")
else:
    print("CUDA (GPU support) is not available. Using CPU.")

# Create a tensor and check the device it's on
x = torch.tensor([1.0])
if x.is_cuda:
    print("x is on GPU.")
else:
    print("x is on CPU.")

Using GPU: NVIDIA GeForce GTX 1080
x is on CPU.


In [ ]:
import os
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import torch
from torch import nn, optim
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

import numpy as np

# from ResTCN import ResTCN
from ResTCN import ResTCN

torch.manual_seed(0)
num_epochs = 100
batch_size = 4
lr = .001
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
print("Device being used:", device, flush=True)


dataloader = get_dataloader(batch_size,
                            'Train.csv',
                            os.path.join(os.getcwd()),
                            'Test.csv',
                            os.path.join(os.getcwd()))
dataset_sizes = {x: len(dataloader[x].dataset) for x in ['train', 'test']}
print(dataset_sizes, flush=True)


model = ResTCN().to(device)
# optimizer = optim.Adam(model.parameters(), lr=lr)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = StepLR(optimizer, step_size=50, gamma=.1)

criterion = nn.CrossEntropyLoss().to(device)
softmax = nn.Softmax()

for epoch in range(num_epochs):

    for phase in ['train', 'test']:

        running_loss = .0
        y_trues = np.empty([0])
        y_preds = np.empty([0])

        if phase == 'train':
            model.train()
        else:
            model.eval()

        for inputs, labels in tqdm(dataloader[phase], disable=True):
            inputs = inputs.to(device)
            labels = labels.long().squeeze().to(device)

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs).squeeze()
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            preds = torch.max(softmax(outputs), 1)[1]
            y_trues = np.append(y_trues, labels.data.cpu().numpy())
            y_preds = np.append(y_preds, preds.cpu())

        # if phase == 'train':
        #     scheduler.step()

        epoch_loss = running_loss / dataset_sizes[phase]

        print("[{}] Epoch: {}/{} Loss: {} LR: {}".format(
            phase, epoch + 1, num_epochs, epoch_loss, scheduler.get_last_lr()), flush=True)
        print('\nconfusion matrix\n' + str(confusion_matrix(y_trues, y_preds)))
        print('\naccuracy\t' + str(accuracy_score(y_trues, y_preds)))

torch.save(model, "model-v2.pt")


Device being used: cuda
{'train': 5698, 'test': 1784}


/tmp/ipykernel_5995/1896814468.py:66: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = torch.max(softmax(outputs), 1)[1]


[train] Epoch: 1/100 Loss: 0.9905852909337516 LR: [0.001]

confusion matrix
[[ 135    1  149   89]
 [  12    0  134   67]
 [  41    2 1592  982]
 [  23    3 1352 1116]]

accuracy	0.49894699894699895


/tmp/ipykernel_5995/1896814468.py:66: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = torch.max(softmax(outputs), 1)[1]


[test] Epoch: 1/100 Loss: 1.1688555027337353 LR: [0.001]

confusion matrix
[[  2   0   2   0]
 [ 22   0  62   0]
 [190   0 692   0]
 [174   0 638   2]]

accuracy	0.3901345291479821


...

/tmp/ipykernel_5995/1896814468.py:66: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = torch.max(softmax(outputs), 1)[1]


[train] Epoch: 25/100 Loss: 0.5743708532550704 LR: [0.001]

confusion matrix
[[ 358    2   11    3]
 [   7   76  117   13]
 [   2   23 2218  374]
 [   3    5  909 1577]]

accuracy	0.7421902421902422


/tmp/ipykernel_5995/1896814468.py:66: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = torch.max(softmax(outputs), 1)[1]


[test] Epoch: 25/100 Loss: 1.0169354619065742 LR: [0.001]

confusion matrix
[[  0   1   2   1]
 [  0   5  70   9]
 [  2   9 759 112]
 [  6  11 669 128]]

accuracy	0.5


/tmp/ipykernel_5995/1896814468.py:66: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = torch.max(softmax(outputs), 1)[1]


In [5]:
torch.version.cuda

'11.5'

In [6]:
model_loaded = torch.load("model-aug.pt") 

In [ ]:
[[  0   1   2   1]
 [  0   2  60  22]
 [  1   3 763 115]
 [  0   0 601 213]]


In [17]:
def calculate_metrics(confusion_matrix):
    # Assuming confusion_matrix is a 2D list of shape (n, n)
    n = len(confusion_matrix)

    # Calculate True Positives, False Positives, False Negatives, and True Negatives
    tp = [confusion_matrix[i][i] for i in range(n)]
    fp = [sum(confusion_matrix[j][i] for j in range(n) if j != i) for i in range(n)]
    fn = [sum(confusion_matrix[i][j] for j in range(n) if j != i) for i in range(n)]
    tn = [sum(confusion_matrix[i][j] for j in range(n) for i in range(n)) - tp[i] - fp[i] - fn[i] for i in range(n)]

    # Calculate Precision, Recall, and Accuracy
    precision = [tp[i] / (tp[i] + fp[i]) if (tp[i] + fp[i]) != 0 else 0 for i in range(n)]
    recall = [tp[i] / (tp[i] + fn[i]) if (tp[i] + fn[i]) != 0 else 0 for i in range(n)]
    accuracy = sum(tp[i] + tn[i] for i in range(n)) / sum(sum(row) for row in confusion_matrix)

    return precision, recall, accuracy

# Example usage:
confusion_matrix = [
    [  0 ,1 ,2 ,1],
    [  0,2,60,22],
    [  1,3,763,115],
    [  0,0,601,213]
]

precision, recall, accuracy = calculate_metrics(confusion_matrix)

print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)


Precision: [0.0, 0.3333333333333333, 0.5350631136044881, 0.6068376068376068]
Recall: [0.0, 0.023809523809523808, 0.8650793650793651, 0.2616707616707617]
Accuracy: 3.096412556053812


In [28]:
import numpy as np

def calculate_metrics(confusion_matrix):
    true_positives = np.diag(confusion_matrix)
    false_positives = np.sum(confusion_matrix, axis=0) - true_positives
    false_negatives = np.sum(confusion_matrix, axis=1) - true_positives
    true_negatives = np.sum(confusion_matrix) - (true_positives + false_positives + false_negatives)

    accuracy = (true_positives + true_negatives) / np.sum(confusion_matrix)
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)

    print(np.sum(true_positives+true_negatives) / np.sum(confusion_matrix) ) 

    return accuracy, precision, recall

In [29]:
calculate_metrics(np.array(confusion_matrix))

3.096412556053812


(array([0.99719731, 0.95179372, 0.56165919, 0.58576233]),
 array([0.        , 0.33333333, 0.53506311, 0.60683761]),
 array([0.        , 0.02380952, 0.86507937, 0.26167076]))

In [ ]:
0.48
0.381

In [32]:
conf_matrix_combines = [
    [   1,0,3],
    [   3,6,75],
    [  13,24,1659]
]

In [33]:
calculate_metrics(np.array(conf_matrix_combines))

2.867713004484305


(array([0.98934978, 0.94282511, 0.93553812]),
 array([0.05882353, 0.2       , 0.95509499]),
 array([0.25      , 0.07142857, 0.97818396]))

In [ ]:
0.40
0.43

In [5]:
print('\naccuracy\t' + str(accuracy_score(y_trues, y_preds)))


accuracy	0.5061659192825112


In [6]:
print('\nconfusion matrix\n' + str(confusion_matrix(y_trues, y_preds)))


confusion matrix
[[  0   0   1   3]
 [  0   2  33  49]
 [  2   0 378 502]
 [  2   1 288 523]]


In [31]:
# y_true_mod 
print('\nconfusion matrix\n' + str(confusion_matrix(y_trues, y_preds)))


confusion matrix
[[  1   0   0   3]
 [  3   6  33  42]
 [  6  14 319 543]
 [  7  10 269 528]]


In [24]:
import numpy as np

y_true_mod = np.copy(y_trues)
y_pred_mod = np.copy(y_preds)

y_true_mod[y_true_mod==3] = 2
y_pred_mod[y_pred_mod==3] = 2

In [25]:
print('\naccuracy\t' + str(accuracy_score(y_true_mod, y_pred_mod)))


accuracy	0.9338565022421524


In [27]:
print('\nconfusion matrix\n' + str(confusion_matrix(y_true_mod, y_pred_mod)))


confusion matrix
[[   1    0    3]
 [   3    6   75]
 [  13   24 1659]]


In [30]:
unique_values, counts = np.unique(y_trues, return_counts=True)

# Combine unique values and counts
unique_with_counts = np.column_stack((unique_values, counts))

print("Unique Values with Counts:")
print(unique_with_counts)

Unique Values with Counts:
[[  0.   4.]
 [  1.  84.]
 [  2. 882.]
 [  3. 814.]]


In [16]:
import os
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import torch
from torch import nn, optim
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

import numpy as np

# from ResTCN import ResTCN
from ResTCN import ResTCN

torch.manual_seed(0)
num_epochs = 1
batch_size = 4
lr = .001
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
print("Device being used:", device, flush=True)


dataloader = get_dataloader(batch_size,
                            'Train.csv',
                            os.path.join(os.getcwd()),
                            'Test.csv',
                            os.path.join(os.getcwd()))
dataset_sizes = {x: len(dataloader[x].dataset) for x in ['train', 'test']}
print(dataset_sizes, flush=True)


model = ResTCN().to(device)
# optimizer = optim.Adam(model.parameters(), lr=lr)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = StepLR(optimizer, step_size=50, gamma=.1)

criterion = nn.CrossEntropyLoss().to(device)
softmax = nn.Softmax()

for epoch in range(num_epochs):

    for phase in ['train', 'test']:

        running_loss = .0
        y_trues = np.empty([0])
        y_preds = np.empty([0])

        if phase == 'train':
            model.train()
        else:
            model.eval()

        for inputs, labels in tqdm(dataloader[phase], disable=True):
            inputs = inputs.to(device)
            labels = labels.long().squeeze().to(device)

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs).squeeze()
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            preds = torch.max(softmax(outputs), 1)[1]
            y_trues = np.append(y_trues, labels.data.cpu().numpy())
            y_preds = np.append(y_preds, preds.cpu())

        # if phase == 'train':
        #     scheduler.step()

        epoch_loss = running_loss / dataset_sizes[phase]

        print("[{}] Epoch: {}/{} Loss: {} LR: {}".format(
            phase, epoch + 1, num_epochs, epoch_loss, scheduler.get_last_lr()), flush=True)
        print('\nconfusion matrix\n' + str(confusion_matrix(y_trues, y_preds)))
        print('\naccuracy\t' + str(accuracy_score(y_trues, y_preds)))

# torch.save(model, "model-v2.pt")


In [12]:
import os
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import torch
from torch import nn, optim
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

import numpy as np

# from ResTCN import ResTCN
from ResTCN import ResTCN

torch.manual_seed(0)
num_epochs = 1
batch_size = 4
lr = .001
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
print("Device being used:", device, flush=True)

dataloader = get_dataloader(batch_size,
                            'Train.csv',
                            os.path.join(os.getcwd()),
                            'Test.csv',
                            os.path.join(os.getcwd()))
dataset_sizes = {x: len(dataloader[x].dataset) for x in ['train', 'test']}
print(dataset_sizes, flush=True)


model = ResTCN().to(device)
# optimizer = optim.Adam(model.parameters(), lr=lr)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = StepLR(optimizer, step_size=50, gamma=.1)

criterion = nn.CrossEntropyLoss().to(device)
softmax = nn.Softmax()

model.train()

phases = ["train","test"]

for phase in phases:
    running_loss = .0
    y_trues = np.empty([0])
    y_preds = np.empty([0])

    if phase == 'train':
        model.train()
    else:
        model.eval()
        
    for inputs, labels in tqdm(dataloader[phase], disable=True):
        inputs = inputs.to(device)
        print("printing input shape")
        print(inputs.shape)
        labels = labels.long().squeeze().to(device)
    
        with torch.set_grad_enabled(phase == 'train'):
            outputs = model(inputs).squeeze()
            print(outputs.shape)
            loss = criterion(outputs, labels)
    
            if phase == 'train':
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
    
        running_loss += loss.item() * inputs.size(0)
        preds = torch.max(softmax(outputs), 1)[1]
        y_trues = np.append(y_trues, labels.data.cpu().numpy())
        y_preds = np.append(y_preds, preds.cpu())
        print("DONE")
        break

Device being used: cuda
{'train': 5698, 'test': 1784}
In init
printing input shape
torch.Size([4, 16, 3, 224, 224])
printing z before transpose
tensor([[[ 2.6490e-01,  6.4568e-01, -7.6464e-01,  ...,  1.2411e+00,
          -4.1636e-01, -8.9891e-01],
         [ 2.3127e-01,  6.1020e-01, -8.0479e-01,  ...,  1.3152e+00,
          -4.3911e-01, -9.0772e-01],
         [ 2.5385e-01,  6.9333e-01, -8.1505e-01,  ...,  1.2990e+00,
          -4.7975e-01, -8.8040e-01],
         ...,
         [ 2.2413e-01,  8.3650e-01, -8.1287e-01,  ...,  1.4418e+00,
          -4.6848e-01, -8.1583e-01],
         [ 1.9923e-01,  7.4005e-01, -7.5658e-01,  ...,  1.3353e+00,
          -4.4813e-01, -7.9885e-01],
         [ 2.0489e-01,  6.7177e-01, -7.4626e-01,  ...,  1.3351e+00,
          -4.2609e-01, -7.8578e-01]],

        [[-9.9288e-01,  2.0541e-01, -7.2315e-01,  ...,  1.3025e+00,
          -5.5616e-01, -1.4155e+00],
         [-1.0772e+00,  2.2200e-01, -7.1238e-01,  ...,  1.2330e+00,
          -5.6015e-01, -1.4154e+00],


/tmp/ipykernel_1951303/260619403.py:69: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = torch.max(softmax(outputs), 1)[1]


DONE
printing input shape
torch.Size([4, 16, 3, 224, 224])
printing z before transpose
tensor([[[-9.0812e-01,  3.2425e-01, -1.1335e+00,  ...,  1.5738e+00,
          -2.2343e-02, -3.0762e-01],
         [-8.7166e-01,  2.7194e-01, -1.1368e+00,  ...,  1.6885e+00,
          -1.9232e-02, -3.0282e-01],
         [-7.7375e-01,  3.2274e-01, -1.1702e+00,  ...,  1.5743e+00,
          -3.5321e-02, -2.4089e-01],
         ...,
         [-1.0045e+00,  4.2009e-01, -1.2452e+00,  ...,  1.6912e+00,
          -1.4274e-01, -1.3128e-01],
         [-9.1295e-01,  4.7431e-01, -1.2426e+00,  ...,  1.6927e+00,
          -5.3225e-02,  1.4605e-03],
         [-8.1339e-01,  4.2340e-01, -1.1836e+00,  ...,  1.6204e+00,
           6.6551e-02, -1.4055e-02]],

        [[-6.9904e-01,  5.8570e-01, -1.4385e+00,  ...,  2.4265e+00,
           6.8996e-02, -9.4630e-01],
         [-6.5677e-01,  5.7836e-01, -1.4893e+00,  ...,  2.4733e+00,
          -6.1878e-02, -9.9188e-01],
         [-6.6010e-01,  5.8102e-01, -1.4218e+00,  ...,  2

In [5]:
vals = torch.randn([4,2,3])
vals

tensor([[[ 2.0338,  0.0901, -0.1384],
         [-0.2649,  0.0132, -0.1543]],

        [[ 0.4264,  0.4951,  0.0973],
         [-0.5939,  0.8031,  0.7879]],

        [[-1.2858,  0.5795,  1.2327],
         [ 0.9387, -1.4228, -1.0053]],

        [[ 1.7012,  2.7311, -0.2984],
         [-1.1312,  0.3651,  0.8213]]])

In [6]:
torch.sum(vals, dim=2)

tensor([[ 1.9854, -0.4060],
        [ 1.0189,  0.9971],
        [ 0.5264, -1.4894],
        [ 4.1340,  0.0552]])

In [7]:
model

ResTCN(
  (tcn): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(32, 128, kernel_size=(7,), stride=(1,), padding=(6,))
        (chomp1): Chomp1d()
        (relu1): ReLU()
        (dropout1): Dropout(p=0.1, inplace=False)
        (conv2): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(6,))
        (chomp2): Chomp1d()
        (relu2): ReLU()
        (dropout2): Dropout(p=0.1, inplace=False)
        (net): Sequential(
          (0): Conv1d(32, 128, kernel_size=(7,), stride=(1,), padding=(6,))
          (1): Chomp1d()
          (2): ReLU()
          (3): Dropout(p=0.1, inplace=False)
          (4): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(6,))
          (5): Chomp1d()
          (6): ReLU()
          (7): Dropout(p=0.1, inplace=False)
        )
        (downsample): Conv1d(32, 128, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock(
        (conv1): Conv1d(128, 128, kernel_size=(7,), 